# Fast RCNN

**Fast R-CNN** es una mejora directa de la R-CNN clásica.  

En este caso, las regiones de interés (*ROIs*) ya no se recortan ni se procesan individualmente para extraer características.  
En su lugar, la imagen completa pasa una sola vez por la red convolucional, generando un **feature map**.  

Después, se obtienen las propuestas de **Selective Search** y se proyectan sus coordenadas sobre ese mapa de características para extraer las regiones correspondientes.  

Dado que cada ROI tiene un tamaño distinto, se aplica **ROI Pooling** para normalizarlas a una forma fija, de modo que puedan pasar por las capas fully-connected del *head*.  (En implementaciones modernas usamos ROI Align)

La red se entrena optimizando simultáneamente:
- la **pérdida de clasificación** (qué objeto hay en cada ROI), y  
- la **pérdida de regresión** (ajuste fino de los *bounding boxes*).  

Durante el entrenamiento **no se aplica NMS**, pero **sí se usa en la inferencia** para eliminar detecciones redundantes.  

Esta arquitectura mejora notablemente los tiempos respecto a R-CNN, aunque **Selective Search sigue siendo el cuello de botella principal**.  


Diferencias:

| Etapa     | R-CNN clásica                  | Fast-RCNN                   |
| --------- | ------------------------------ | ------------------------------------------- |
| Proposals | Selective Search               | Selective Search                 |
| ROIs      | No hay como tal                | Subconjunto de proposals usado por ROIPooling / Align |
| NMS       | Final (sobre predicciones SVM) | Final (sobre predicciones del head)         |


Conceptos:


| Concepto      | Qué es                                                    | Cómo se obtiene                                        | Cuántos hay              | Cómo se usa                                                |
| ------------- | --------------------------------------------------------- | ------------------------------------------------------ | ------------------------ | ---------------------------------------------------------- |
| **Proposals** | Candidatas “donde podría haber algo”                      | De un algoritmo como **Selective Search** o un **RPN** | Miles (≈2000 por imagen) | Entrada bruta del detector                                 |
| **ROIs**      | Subconjunto de proposals elegidas para entrenar o inferir | Se **muestrean** (pos/neg) de las proposals            | Pocas (≈128 por imagen)  | Alimentan **ROIAlign + head** para clasificación/regresión |


Vamos a ver su implementación

In [64]:
import os, random, math, time, pathlib, shutil, json
import numpy as np
import torch
import torchvision as tv
import selectivesearch
from torchvision.models import ResNet50_Weights
from torchsummary import summary
from torchvision.datasets import VOCDetection
from torchvision.transforms.functional import to_tensor, normalize

weights = ResNet50_Weights.IMAGENET1K_V2
resnet_transforms = weights.transforms()


torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("CUDA disponible:", torch.cuda.is_available())
print("Device:", device)

VOC_ROOT = os.environ.get("VOC_ROOT", "./data/02")
print("VOC_ROOT:", VOC_ROOT)


CUDA disponible: True
Device: cuda
VOC_ROOT: ./data/02


#### Nos saltamos la parte de descargar el dataset, lo vimos en el cuaderno anterior.

Pasamos directamente a su carga.

In [65]:


train_root = "./data/02/VOCtrainval_06-Nov-2007"
test_root  = "./data/02/VOCtest_06-Nov-2007"

train_ds = VOCDetection(train_root, year="2007", image_set="trainval", download=False)
test_ds  = VOCDetection(test_root,  year="2007", image_set="test",     download=False)

print("Train:", len(train_ds))
print("Test:", len(test_ds))


Train: 5011
Test: 4952


In [66]:
# === Extracción automática de clases del dataset VOC ===

def extract_voc_classes(dataset):
    classes = set()
    for i in range(min(500, len(dataset))):  # escanea solo 500 imágenes para acelerar
        ann = dataset[i][1]['annotation']
        objs = ann.get('object', [])
        if isinstance(objs, dict):  # si solo hay un objeto
            objs = [objs]
        for obj in objs:
            name = obj['name']
            classes.add(name)
    return sorted(list(classes))

# Obtener clases del dataset de entrenamiento
voc_classes = extract_voc_classes(train_ds)

#Añadimos background porque es la clase 0, que no está en voc_classes, lo necesitamos para el loss. 
#Tendremos 21 clases: 20 de VOC + 1 de background.
CLASSES = ['background'] + voc_classes

NUM_CLASSES = len(CLASSES)

print("Clases detectadas:", voc_classes)
print("Total:", NUM_CLASSES)


Clases detectadas: ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']
Total: 21


Tenemos unas 2000 proposals por imagen
Estas salen de Selective Search.
La mayoría no contiene ningún objeto útil; algunas sí cubren bien GT.
El objetivo es quedarse con 128 ROIs representativas para esa imagen.

Para ello calculamos el IoU entre cada proposal y todas las cajas GT.

Luego clasificamos cada proposal en Pos y Neg en base a los thresholds del IoU y poderamos las imagenes que nos queremos quedar.


In [67]:


# IoU thresholds
POS_IOU_THRESH = 0.5
NEG_IOU_THRESH = 0.3

# Sampler config
# Tomamos 128 ROIs por imagen.
ROIS_PER_IMG = 128
# 25% de esas ROIs son positivas (foreground) y el resto 75% negativas (background).
# Usamos esta distribución porque es lo que da mejores resultados en la literatura.
FG_FRACTION = 0.25  # ~32 pos + 96 neg 

# BBox delta normalization
# Normalizamos las coordenadas de los bounding boxes para que sean más estables.
#Son valores que usamos fijos para todo el entrenamiento y que se han obtenido empíricamente.
#Son un estándar en la literatura.
BBOX_MEANS = torch.tensor([0.0, 0.0, 0.0, 0.0], dtype=torch.float32)
BBOX_STDS  = torch.tensor([0.1, 0.1, 0.2, 0.2], dtype=torch.float32)

# Proposals
# Tomamos hasta 2000 propuestas por imagen.
MAX_PROPOSALS_PER_IMG = 2000
# Las propuestas mínimas tienen 16px de lado.
MIN_SIZE = 16

print(f"Num clases: {NUM_CLASSES}")
print(f"Sampler: {ROIS_PER_IMG} (FG {int(ROIS_PER_IMG*FG_FRACTION)}, BG {ROIS_PER_IMG-int(ROIS_PER_IMG*FG_FRACTION)})")
print(f"PosIoU≥{POS_IOU_THRESH}, NegIoU≤{NEG_IOU_THRESH}")

Num clases: 21
Sampler: 128 (FG 32, BG 96)
PosIoU≥0.5, NegIoU≤0.3


Ahora necesitamos redimensionar las imagenes, mantendremos un tamaño fijo para el lado más corto (600) y uno variable de maximo 1000 para el lado largo, de desta manera podemos mantener el factor de escala, este facto de escala tendremos que aplicarlo también a los BBs para que la proyección sea coherente.

In [68]:

from PIL import Image

#Definimos el tamaño fijo para el lado más corto y el máximo para el lado largo
short_side = 600
max_side = 1000

#Obtenemos la media y la desviación estándar de ResNet para la normalización
R50_MEAN = resnet_transforms.mean
R50_STD = resnet_transforms.std
print(R50_MEAN, R50_STD)

#Llamaremos a esta función para preprocesar las imágenes
def preprocess_image(img, boxes, device=device, short_side=short_side, max_side=max_side):
    
    #Objeto PIL de la imagen
    w, h = img.size #Ej (500, 333)
    #Obtenemos el tamaño más corto y el más largo de la imagen
    actual_short = min(w, h) #Ej 333
    actual_long  = max(w, h) #Ej 500

    scale_multiplier = short_side / actual_short
    
    target_long = actual_long * scale_multiplier

    if target_long > max_side:
        scale_multiplier = max_side / actual_long #porque max_side = actual_long * scale_multiplier

    #Calculamos el nuevo tamaño de la imagen
    w = int(w * scale_multiplier)
    h = int(h * scale_multiplier)

    #Redimensionamos la imagen
    img = img.resize((w, h), Image.BILINEAR)

    #Ajustamos los BBs
    #Los BBs vienen en formato (xmin, ymin, xmax, ymax)
    #Es un tensor de 4 columnas, cada fila es un BB
    #Multiplicamos cada columna por el factor de escala
    boxes = boxes * scale_multiplier

    #Convertimos la imagen a tensor
    img = to_tensor(img)

    #Normalizamos la imagen 
    img = normalize(img, R50_MEAN, R50_STD)

    #Los boxes NO se normalizan, se mantienen en las coordenadas originales porque son relativas a la imagen.
    
    return img, boxes, scale_multiplier



[0.485, 0.456, 0.406] [0.229, 0.224, 0.225]


Bien, ahora, necesitamos una funcion que reciba una imagen  y extraiga las bbs de esa imagen en un tensor.

In [69]:
#Esto es puro parseo de XML, no hay nada especial.
def extract_gt_boxes(annotation):
    objs = annotation["annotation"]["object"]
    
    if isinstance(objs, dict):
        objs = [objs]  

    boxes = []
    for o in objs:
        bb = o["bndbox"]
        xmin = float(bb["xmin"])
        ymin = float(bb["ymin"])
        xmax = float(bb["xmax"])
        ymax = float(bb["ymax"])
        boxes.append([xmin, ymin, xmax, ymax])

    return torch.tensor(boxes, dtype=torch.float32)

        

Vamos a probar con una unica imagen.

In [70]:
img, ann = train_ds[0]

#Extraemos las bbs de la imagen
boxes = extract_gt_boxes(ann)

print("Tamaño original:", img.size)
print("Boxes originales:", boxes)

#Preprocesamos la imagen y los boxes
img_t, boxes_t, scale = preprocess_image(img, boxes)

print("Escala aplicada:", scale)
print("Tamaño nuevo:", img_t.shape)
print("Boxes reescaladas:", boxes_t)


Tamaño original: (500, 375)
Boxes originales: tensor([[263., 211., 324., 339.],
        [165., 264., 253., 372.],
        [  5., 244.,  67., 374.],
        [241., 194., 295., 299.],
        [277., 186., 312., 220.]])
Escala aplicada: 1.6
Tamaño nuevo: torch.Size([3, 600, 800])
Boxes reescaladas: tensor([[420.8000, 337.6000, 518.4000, 542.4000],
        [264.0000, 422.4000, 404.8000, 595.2000],
        [  8.0000, 390.4000, 107.2000, 598.4000],
        [385.6000, 310.4000, 472.0000, 478.4000],
        [443.2000, 297.6000, 499.2000, 352.0000]])


Ahora tendremos que crear una funcion que calcule los proposals con selective search de una imagen ya preprocesada

In [71]:


def get_selective_proposals(img_pil, max_proposals=MAX_PROPOSALS_PER_IMG, min_size=MIN_SIZE):
    img_np = np.array(img_pil)

    _, regions = selectivesearch.selective_search(
        img_np,
        scale=500,
        sigma=0.9,
        min_size=10
    )

    seen = set()
    proposals = []

    for r in regions:
        x, y, w, h = r["rect"]

        if (x, y, w, h) in seen:
            continue
        seen.add((x, y, w, h))

        if w < min_size or h < min_size:
            continue

        proposals.append([x, y, x + w, y + h])

        if len(proposals) >= max_proposals:
            break

    if len(proposals) == 0:
        return torch.zeros((0, 4), dtype=torch.float32)

    return torch.tensor(proposals, dtype=torch.float32)
